In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 13]

In [3]:
station['date'].count()

1035

In [4]:
# ls == 모든 값이 0인 것

ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

(['depart', 'sunrise', 'sunset', 'snowfall', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'snowfall', 'daytime'])

In [5]:
for i in station.columns:
    count_null = station[i].isna().sum()
    print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

station_nbr : 0 ( 0.0 % )
date : 0 ( 0.0 % )
tmax : 0 ( 0.0 % )
tmin : 0 ( 0.0 % )
tavg : 0 ( 0.0 % )
depart : 1035 ( 100.0 % )
dewpoint : 0 ( 0.0 % )
wetbulb : 0 ( 0.0 % )
heat : 0 ( 0.0 % )
cool : 0 ( 0.0 % )
sunrise : 1035 ( 100.0 % )
sunset : 1035 ( 100.0 % )
codesum : 0 ( 0.0 % )
snowfall : 1035 ( 100.0 % )
preciptotal : 0 ( 0.0 % )
stnpressure : 0 ( 0.0 % )
sealevel : 0 ( 0.0 % )
resultspeed : 0 ( 0.0 % )
resultdir : 0 ( 0.0 % )
avgspeed : 0 ( 0.0 % )
year : 0 ( 0.0 % )
month : 0 ( 0.0 % )
day : 0 ( 0.0 % )
relative_humility : 0 ( 0.0 % )
windchill : 0 ( 0.0 % )
daytime : 1035 ( 100.0 % )
week7 : 0 ( 0.0 % )
weekend : 0 ( 0.0 % )
code_change : 0 ( 0.0 % )
rainY : 0 ( 0.0 % )
otherY : 0 ( 0.0 % )
nothing : 0 ( 0.0 % )


In [6]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [7]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [8]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
561100,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,NaN,4,0,0,0,0,1,40,107,0
561101,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,NaN,4,0,0,0,0,1,40,108,0
561102,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,NaN,4,0,0,0,0,1,40,109,0
561103,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,NaN,4,0,0,0,0,1,40,110,0
561104,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,NaN,4,0,0,0,0,1,40,111,0


In [9]:
station['log1p_units'] = np.log1p(station.units)

In [10]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
561100,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,42.873464,NaN,4,0,0,0,0,1,40,107
561101,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,42.873464,NaN,4,0,0,0,0,1,40,108
561102,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,42.873464,NaN,4,0,0,0,0,1,40,109
561103,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,42.873464,NaN,4,0,0,0,0,1,40,110
561104,13,2014-10-31,58.0,35.0,46.5,NaN,28.0,37.0,18.0,0.0,...,42.873464,NaN,4,0,0,0,0,1,40,111


In [11]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [12]:
df2.to_csv('station13.csv', sep=',', index=False)

In [13]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 있음
['depart', 'sunrise', 'sunset', 'snowfall', 'daytime']


In [14]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

In [15]:
ls, ls1

(['depart', 'sunrise', 'sunset', 'snowfall', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'snowfall', 'daytime'])

### 1. 변수변환: df2 (log1p_units)

In [19]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + C(daytime) + 0'

In [16]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.704
Model:                            OLS   Adj. R-squared:                  0.704
Method:                 Least Squares   F-statistic:                     7544.
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:39:48   Log-Likelihood:            -1.2232e+05
No. Observations:              561105   AIC:                         2.450e+05
Df Residuals:                  560927   BIC:                         2.470e+05
Df Model:                         177                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [17]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

15186

In [18]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,1,0.0
1,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,2,0.0
2,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,3,0.0
3,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,4,0.0
4,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,6,0.0
5,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,7,0.0
6,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,8,0.0
7,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,10,0.0
8,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,11,0.0
9,13,2012-01-01,38.0,18.0,28.0,NaN,12.0,21.0,37.0,0.0,...,NaN,6,1,0,0,0,1,15,12,0.0


In [19]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 2.685e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:45:34   Log-Likelihood:             9.9667e+05
No. Observations:              545919   AIC:                        -1.993e+06
Df Residuals:                  545741   BIC:                        -1.991e+06
Df Model:                         177                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

### 3. VIF 확인

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill' ]

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF Factor,features
0,inf,tmax
1,inf,tmin
2,inf,tavg
3,1.506697e+06,stnpressure
4,1.461076e+06,sealevel
5,4.885527e+03,windchill
6,1.426444e+03,wetbulb
7,2.575549e+02,dewpoint
8,1.439019e+02,relative_humility
9,1.365119e+02,heat


### 4. VIF 근거, tmax, tmin, tavg, stnpre, sea, wind, wetbult, dewpoint, relative, heat 빼고 다시 Model 돌림.

In [23]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(dewpoint) + scale(cool)\
                            + scale(preciptotal) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 2.812e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        12:06:37   Log-Likelihood:             9.9666e+05
No. Observations:              545919   AIC:                        -1.993e+06
Df Residuals:                  545749   BIC:                        -1.991e+06
Df Model:                         169                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(resultdir)[1.0]      0.0034      0

In [26]:
ls1

['depart', 'sunrise', 'sunset', 'snowfall', 'daytime']

### 5. VIF 재확인 (tmax, tmin, tavg 제외함)

In [27]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['dewpoint','cool','preciptotal',\
        'resultspeed','avgspeed']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,14.456026,avgspeed
1,8.128052,resultspeed
2,5.014932,dewpoint
3,1.417994,cool
4,1.134908,preciptotal


### 6. VIF 근거, avg speed 제외하고  Model 돌림.

In [28]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(dewpoint) + scale(cool)\
                            + scale(preciptotal) + scale(resultspeed)  \
                            + C(resultdir) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 2.828e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        12:12:17   Log-Likelihood:             9.9666e+05
No. Observations:              545919   AIC:                        -1.993e+06
Df Residuals:                  545750   BIC:                        -1.991e+06
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(resultdir)[1.0]      0.0035      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [29]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(resultspeed),1.0,0.012565,0.012565,8.264788,4.042294e-03
C(rainY),1.0,0.047129,0.047129,31.000255,2.581154e-08
C(weekend),1.0,0.474252,0.474252,311.952155,8.579099e-70
C(month),11.0,0.803121,0.073011,48.024923,3.366251e-106
C(year),2.0,1.679739,0.839870,552.446620,2.079805e-240
C(store_nbr),4.0,1.952923,0.488231,321.146764,1.551068e-276
C(resultdir),36.0,628.161302,17.448925,11477.494462,0.000000e+00
C(item_nbr),110.0,72233.031193,656.663920,431938.155491,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [30]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(resultspeed)  \
                            + C(resultdir) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 2.880e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        12:16:24   Log-Likelihood:             9.9666e+05
No. Observations:              545919   AIC:                        -1.993e+06
Df Residuals:                  545753   BIC:                        -1.991e+06
Df Model:                         165                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(resultdir)[1.0]      0.0034      0

In [31]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


dfX = df2[['year', 'month', 'weekend', 'rainY', 'store_nbr', 'item_nbr', 'resultdir', 'resultspeed']]
dfy = df2['log1p_units']

model = LinearRegression()

cv = KFold(10, shuffle=True)

cross_val_score(model, dfX, dfy, scoring="r2", cv=cv)

array([0.01402272, 0.01390284, 0.01402932, 0.01573242, 0.01621126,
       0.01548514, 0.01520941, 0.01328605, 0.01397981, 0.01452603])

In [32]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [33]:
formula = 'C(year) + C(month) + C(weekend) + C(resultdir) \
        + C(rainY) + C(store_nbr) + C(item_nbr) + scale(resultspeed) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.98665732, 0.98702967, 0.98836085, 0.98355483, 0.99107122,
       0.99017401, 0.98870186, 0.99002734, 0.99141449, 0.98789855])